# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [57]:
# import libraries
import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import os
import pickle


pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
pd.set_option('display.max_colwidth', -1)


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.base import BaseEstimator,TransformerMixin

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# load data from database

# load data from database
database_filepath = "DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_messages_clean"
df = pd.read_sql_table(table_name,engine)


In [5]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.",direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
for i in df.columns.tolist():
    print(i,df[i].unique())

id [    2     7     8 ..., 30109 30110 30111]
message ['Weather update - a cold front from Cuba that could pass over Haiti'
 'Is the Hurricane over or is it not over'
 'Looking for someone but no name' ...,
 'Damage to the Nacala rail corridor by the rains has delayed expected arrivals at least by seven days.'
 'Thus, an armed drone strike in Pakistan, a Nigerian Army massacre of villagers, plans for a new US drone base in Niger ,or joint US/Israeli military operations, are all seen as part of a wider pattern of western oppression, persistently spread through the new social media."'
 'This year\'s 16 Days theme, "From Peace in the Home to Peace in the World," focuses on the role of militarism, and highlights how "war, internal conflict, and violent repression have a particular and often disproportionate impact on women and the violence they experience."']
original [ 'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encor

In [7]:
#notice that related column has 3 categories, lets move 2 into 1 as that must be a typo
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [8]:
X = df['message']
Y = df.iloc[:,4:]

In [9]:
Y.shape

(26207, 36)

In [10]:
X.isnull().sum()

0

### 2. Write a tokenization function to process your text data

In [11]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [48]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
        # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:


pipeline= Pipeline([('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_fit = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline_fit.predict(X_test)

In [16]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
   
    print(column)
    print(classification_report(y_test[column],y_pred_pd[column]))

related
             precision    recall  f1-score   support

        0.0       0.35      0.13      0.19      1519
        1.0       0.78      0.93      0.85      5033

avg / total       0.68      0.74      0.69      6552

request
             precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      5403
        1.0       0.38      0.06      0.11      1149

avg / total       0.75      0.82      0.76      6552

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6523
        1.0       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.59      0.82      0.69      3791
        1.0       0.45      0.20      0.28      2761

avg / total       0.53      0.56      0.51      6552

medical_help
             precision    recall  f1-score   support

        0.0       0.92      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...tfidf_transformer', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('classifier',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
       

In [21]:
# Number of trees in random forest
n_estimators = [200, 500]
# Number of features to consider at every split
#max_features = ['auto', 'sqrt']

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid

parameters = { 'features__text_pipeline__count_vectorizer__max_features': (None, 5000),
                'features__text_pipeline__tfidf_transformer__use_idf': (True, False)
              #,
               # 'classifier__estimator__n_estimators': n_estimators
              #,
              # 'classifier__estimator__max_features': max_features,
               
              # 'classifier__estimator__min_samples_split': min_samples_split
              #,
               #'classifier__estimator__min_samples_leaf': min_samples_leaf
    #,
 #              'classifier__estimator__bootstrap': bootstrap
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv= None)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__count_vectorizer__max_features': (None, 5000), 'features__text_pipeline__tfidf_transformer__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
# Get the prediction values from the grid search cross validator
y_pred = cv.predict(X_test)
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
for column in y_test.columns:
   
    print(column)
    print(classification_report(y_test[column],y_pred_pd[column]))

related
             precision    recall  f1-score   support

        0.0       0.34      0.11      0.17      1519
        1.0       0.78      0.93      0.85      5033

avg / total       0.68      0.74      0.69      6552

request
             precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      5403
        1.0       0.32      0.05      0.09      1149

avg / total       0.74      0.81      0.75      6552

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6523
        1.0       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.59      0.84      0.69      3791
        1.0       0.46      0.18      0.26      2761

avg / total       0.53      0.57      0.51      6552

medical_help
             precision    recall  f1-score   support

        0.0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [49]:
#IMPLEMENT STARTING VERB EXTRACTOR -TAKES FIRST WORD OF A SENTENCE-SOMETHING WE LEARNED AS PART OF SYLLABUS
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        text = text.replace(" ", "_")
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            #words = word_tokenize(text)
            pos_tags= nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
       
        X_tagged = pd.Series(X).apply(self.starting_verb)
        
        return pd.DataFrame(X_tagged)


In [54]:
#NEW PIPELINE WITH Gradientboost
pipeline_new= Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_extractor', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [55]:
#Use new pipeline
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_fitted = pipeline_new.fit(X_train, y_train)


y_pred = cv.predict(X_test)
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
   
    print(column)
    print(classification_report(y_test[column],y_pred_pd[column]))

related
             precision    recall  f1-score   support

        0.0       0.92      0.73      0.81      1491
        1.0       0.93      0.98      0.95      5061

avg / total       0.92      0.92      0.92      6552

request
             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96      5396
        1.0       0.96      0.65      0.78      1156

avg / total       0.94      0.93      0.93      6552

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6528
        1.0       1.00      0.50      0.67        24

avg / total       1.00      1.00      1.00      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.84      0.95      0.89      3802
        1.0       0.92      0.75      0.82      2750

avg / total       0.87      0.87      0.86      6552

medical_help
             precision    recall  f1-score   support

        0.0       0.96      1.00      0

### 9. Export your model as a pickle file

In [58]:
# save the model to disk
modelpickle = pickle.dumps('multiclassifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.